In [42]:
import requests
import json
import time
import numpy as np 
import sqlite3
import pandas as pd
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup

# 0. Consulta Mapa Actualizado 

In [148]:
A = requests.get("https://www.boliviasegura.gob.bo")
page = BeautifulSoup(A.text, 'html.parser')
iframe = page.find_all("iframe")[0]

iframe.attrs["src"]

In [150]:
from selenium import webdriver
import re

In [161]:
browser = webdriver.Firefox()
browser.get(iframe.attrs["src"])

A = browser.find_element_by_tag_name("img")

time.sleep(2)
lista_dir = []
for img in browser.find_elements_by_tag_name("img"):
    lista_dir = lista_dir + re.findall(".*juliael@(.*)/\d,", img.get_attribute("src"))

mapa_id = np.unique(np.array(lista_dir))[0] # Todos las rutas deberìan referenciar al mismo mapa 
print(mapa_id)
browser.close()

293a2875@a60cb7b38355a207a67a1ea6c9152373:1588040459439


In [162]:
nro_mun = requests.get("https://cartocdn-gusc-a.global.ssl.fastly.net/juliael/api/v1/map/juliael@{mapa_id}/dataview/eac18df0-661c-4c8f-a7c6-532c7ed3b5bf".format(mapa_id=mapa_id))
nro_mun.json()["count"]

49

In [160]:
# Si existe un nuevo mapa se lo consulta

# 1. Consulta MUNICIPIOS

In [136]:
responses = []
for i in tqdm(range(400)):
    registro_i = requests.get("https://cartocdn-gusc-c.global.ssl.fastly.net/juliael/api/v1/map/juliael@{mapa_id}/4/attributes/{mun_id}".format(mapa_id = mapa_id, mun_id= i))
    responses.append([i, registro_i.headers["Date"] , registro_i])
    time.sleep(0.5)

VALUES = {}
for response_i in responses: 
    if response_i[2].ok:
        dict_i = {}
        dict_i["date_get"] =  response_i[1]
        dict_i.update(response_i[2].json())
        VALUES[str(response_i[0])] = dict_i

VALUES.keys()

CONSULTA_i = pd.DataFrame(VALUES).T.reset_index()
CONSULTA_i["date_get"] = pd.to_datetime(CONSULTA_i.date_get)
CONSULTA_i.fillna(value=0, inplace=True)
CONSULTA_i["activos"] = CONSULTA_i.positivos - CONSULTA_i.decesos - CONSULTA_i.recuperados

CONSULTA_i.to_csv("{} Municipios.csv".format(CONSULTA_i.date_get.max().strftime("%Y-%m-%d %H:%M")),index=False)

